This code did data cleaning, computed KNDVI, and matrix profile.
Users are advised to pay attention to all sections of the code.

Last update Thursday Oct 31 2024
@author: Pius N.Nwachukwu

In [ ]:
!pip install jupyter_bokeh


In [ ]:
!pip install chord

In [ ]:
pip install holoviews bokeh


In [ ]:
!pip install stumpy

In [ ]:
import os
import shutil
import ee
import geemap
import pandas as pd
from google.colab import drive


In [ ]:
# Authenticate and initialize the Earth Engine module.
ee.Authenticate()
ee.Initialize(project='ee-')# Please provide a project name

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Calling up the Saved Time Series NDVI Values
# csv_file_path = '/content/drive/My Drive/FoRes/gq_ndvi_timeseries_data.csv' # Get the NDVIs
csv_file_path = '/content/drive/My Drive/FoRes/ndvi_pixelz_data.csv' # Get the NDVIs in 5000 SR Pixels


# Read the CSV file into a pandas DataFrame
ndvi_df = pd.read_csv(csv_file_path)
print(ndvi_df.head())

        ROI        date  percentage_valid  latitude  longitude  NDVI
0  S_Americ  2000-02-01          29.70645 -6.609515 -58.598977  1355
1  S_Americ  2000-02-01          29.70645 -6.609515 -58.508443  4857
2  S_Americ  2000-02-01          29.70645 -6.654481 -58.694867  3963
3  S_Americ  2000-02-01          29.70645 -6.654481 -58.649596  1773
4  S_Americ  2000-02-01          29.70645 -6.654481 -58.604325  2665


In [ ]:
ndvi_df.set_index('date', inplace=True)

In [ ]:
# Converting the NDVI to Reflectance
ndvi_df["NDVI_2"] = ndvi_df["NDVI"] / 10000

In [ ]:
# # Calculate KNDVI
ndvi_df['KNDVI'] = (ndvi_df['NDVI_2'] + 1) / 2

# print(ndvi_df)

In [ ]:
# ndvi_df['ROI']

# Print all the names of the ROIs
print(ndvi_df['ROI'].unique())

['S_Americ' 'N_Americ' 'Afric' 'Europ' 'Asia' 'Aust_Ocean']


In [ ]:
# Filter the DataFrame to get rows where the 'Region' column is correspnd to a continent eg 'Africa'
af = ndvi_df[ndvi_df["ROI"] == "Afric"]
asia = ndvi_df[ndvi_df["ROI"] == "Asia"]
eu = ndvi_df[ndvi_df["ROI"] == "Europ"]
na = ndvi_df[ndvi_df["ROI"] == "N_Americ"]
sa = ndvi_df[ndvi_df["ROI"] == "S_Americ"]
oc = ndvi_df[ndvi_df["ROI"] == "Aust_Ocean"]
# print(eu)

In [ ]:
# Define the ROI name you want to filter
selected_roi = 'Europ'  # Replace current continent/ROI

# Filter the DataFrame for pixels within the selected ROI
df_filtered = ndvi_df[ndvi_df['ROI'] == selected_roi]

# Display the first few rows of the filtered DataFrame
print(df_filtered.head())

# # Optionally, save the filtered DataFrame to a CSV file
# df_filtered.to_csv(f'{selected_roi}_ndvi_pixel_data.csv', index=False)


In [ ]:
# Rounding the coordinates to group nearby coordinates to effectively clusters locations within a certain decimal precision
#
df_filtered['latitude'] = df_filtered_copy['latitude'].round(2)
df_filtered['longitude'] = df_filtered_copy['longitude'].round(2)
df_filtered



In [ ]:

# Group the data by latitude and longitude to count the unique pixels
unique_pixels = df_filtered.groupby(['latitude', 'longitude']).size()

# Print the number of unique pixels
print(f"Number of unique pixels: {len(unique_pixels)}")

# If you want to see a sample of the pixel data
print(unique_pixels.head())

In [ ]:
# Confirm Pixel Characteristics

# Define the range for valid KNDVI values
valid_ndvi_min = -1
valid_ndvi_max = 1

# Filter the dataset for valid NDVI values
valid_ndvi_data = df_filtered[(df_filtered['NDVI_2'] >= valid_ndvi_min) & (df_filtered['NDVI_2'] <= valid_ndvi_max)]

# Group by latitude and longitude to identify unique pixels with valid data
valid_pixels = valid_ndvi_data.groupby(['latitude', 'longitude']).size()

# Calculate the percentage of valid pixels
percentage_valid_pixels = (len(valid_pixels) / 1277) * 100  # Assuming 5460 is the total number of unique pixels

print(f"Percentage of pixels with valid NDVI data: {percentage_valid_pixels:.2f}%")


In [ ]:
# Reshapes the data so that each row represents a unique pixel (defined by latitude and longitude)

# Pivot the data to create a time series for each pixel
ndvi_pivot = df_filtered.pivot_table(index=['latitude', 'longitude'], columns='date', values='KNDVI')

# Display the reshaped DataFrame
print(ndvi_pivot.head())

# Now, check the length of the time series for one pixel
print(f"Number of time points: {ndvi_pivot.shape[1]}")

In [ ]:
# df_filtered.index

In [ ]:
# Compute MP with Stumpy (With zero Exclusion)

import pandas as pd
import stumpy

# Fill missing values using interpolation or fill with mean (as needed)
ndvi_pivot_filled = ndvi_pivot.interpolate(method='linear', axis=1).fillna(ndvi_pivot.mean(axis=1))

# Initialize a list to store results
results = []

# Loop through each pixel (each row in the pivot table)
for index, row in ndvi_pivot_filled.iterrows():
    # Extract the NDVI time series for this pixel
    ndvi_series = row.values

    # Define a reasonable window size
    window_size = 24

    # Compute the matrix profile for the time series
    mp = stumpy.stump(ndvi_series, m=window_size)

    # Identify the motif (minimum matrix profile value) and its nearest neighbor
    motif_idx = mp[:, 0].argmin()
    distance = mp[motif_idx, 0]  # Matrix profile distance
    neighbor_idx = int(mp[motif_idx, 1])

    # Only proceed if the distance is greater than zero (exclude exact matches)
    if distance > 0:
        # Identify the discord (maximum matrix profile value)
        discord_idx = mp[:, 0].argmax()

        # Extract the dates corresponding to the motif, neighbor, and discord
        motif_date = ndvi_pivot_filled.columns[motif_idx]
        neighbor_date = ndvi_pivot_filled.columns[neighbor_idx]
        discord_date = ndvi_pivot_filled.columns[discord_idx]

        # Append results for this pixel to the list
        results.append({
            'latitude': index[0],
            'longitude': index[1],
            'motif_date': motif_date,
            'neighbor_date': neighbor_date,
            'discord_date': discord_date,
            'motif_value': ndvi_series[motif_idx],
            'neighbor_value': ndvi_series[neighbor_idx],
            'discord_value': ndvi_series[discord_idx],
            'distance': distance  # Include the distance in the results
        })

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# # Save the results to a CSV file
# results_df.to_csv('ndvi_motifs_discords_excluding_zero_distance.csv', index=False)

# # Display the first few rows of the results
# print(results_df.head())


In [ ]:
# Plotting  data point (month).


import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

# Ensure that the date columns are in datetime format
ndvi_pivot_filled.columns = pd.to_datetime(ndvi_pivot_filled.columns)

# Select a pixel (e.g., the first pixel in the pivot table)
selected_pixel_series = ndvi_pivot_filled.iloc[3].values  # NDVI time series for the selected pixel
selected_pixel_index = ndvi_pivot_filled.index[3]  # Get the corresponding latitude and longitude

# Define a reasonable window size
window_size = 24

# Compute the matrix profile for the selected pixel
mp = stumpy.stump(selected_pixel_series, m=window_size)

# Identify the motif, neighbor, and discord indices
motif_idx = mp[:, 0].argmin()
neighbor_idx = int(mp[motif_idx, 1])
discord_idx = mp[:, 0].argmax()

# Plot the NDVI time series
plt.figure(figsize=(7, 4))

# NDVI Time Series
plt.subplot(2, 1, 1)
plt.plot(ndvi_pivot_filled.columns, selected_pixel_series, label='NDVI Time Series')

# Highlight motif, neighbor, and discord
plt.plot(ndvi_pivot_filled.columns[motif_idx:motif_idx+window_size], selected_pixel_series[motif_idx:motif_idx+window_size], color='green', label='Motif')
plt.plot(ndvi_pivot_filled.columns[neighbor_idx:neighbor_idx+window_size], selected_pixel_series[neighbor_idx:neighbor_idx+window_size], color='orange', label='Neighbor')
plt.plot(ndvi_pivot_filled.columns[discord_idx:discord_idx+window_size], selected_pixel_series[discord_idx:discord_idx+window_size], color='red', label='Discord')

plt.title(f'KNDVI Time Series for Nth America at Latitude {selected_pixel_index[0]}, Longitude {selected_pixel_index[1]}')
plt.xlabel('Date')
plt.ylabel('KNDVI')
plt.xticks(rotation=45)

# Set major ticks to the first month of each year
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Matrix Profile
plt.subplot(2, 1, 2)
plt.plot(ndvi_pivot_filled.columns[window_size-1:], mp[:, 0], label='Matrix Profile', color='orange')

# Highlight motif, neighbor, and discord in the matrix profile
plt.scatter(ndvi_pivot_filled.columns[motif_idx + window_size - 1], mp[motif_idx, 0], color='green', label='Motif')
plt.scatter(ndvi_pivot_filled.columns[neighbor_idx + window_size - 1], mp[neighbor_idx, 0], color='orange', label='Neighbor')
plt.scatter(ndvi_pivot_filled.columns[discord_idx + window_size - 1], mp[discord_idx, 0], color='black', label='Discord')

plt.title('Matrix Profile')
plt.xlabel('Date')
plt.ylabel('Matrix Profile Value')
plt.xticks(rotation=45)

# Set major ticks to the first month of each year
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Adjust the layout and place the legend below the plots
plt.tight_layout(rect=[0, 0, 1, 1.2])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.70), ncol=4)
# 0.5, 0.1
plt.show()


In [ ]:
# Compute MP and Reurn the Year Occurence of Motif and Discord

import pandas as pd
from collections import defaultdict
import stumpy

# Create dictionaries to store motif and discord counts by year
motif_counts_by_year = defaultdict(int)
discord_counts_by_year = defaultdict(int)

# Loop through the motif-neighbor pairs and count occurrences by year
for index, row in ndvi_pivot_filled.iterrows():
    ndvi_series = row.values
    dates = row.index  # Assuming the index of the row contains the dates

    window_size = 24
    mp = stumpy.stump(ndvi_series, m=window_size)

    # Identify the motif (minimum matrix profile value) and its nearest neighbor
    motif_idx = mp[:, 0].argmin()
    motif_distance = mp[motif_idx, 0]  # Matrix profile distance for motif

    # Only count motifs where the distance is greater than zero
    if motif_distance > 0:
        motif_year = pd.to_datetime(dates[motif_idx]).year  # Extract year from the date
        motif_counts_by_year[motif_year] += 1  # Increase count for this year

    # Identify the discord (maximum matrix profile value)
    discord_idx = mp[:, 0].argmax()
    discord_distance = mp[discord_idx, 0]  # Matrix profile distance for discord

    # Only count discords where the distance is greater than zero
    if discord_distance > 0:
        discord_year = pd.to_datetime(dates[discord_idx]).year  # Extract year from the date
        discord_counts_by_year[discord_year] += 1  # Increase discord count for this year

# Convert motif counts to a DataFrame for easier manipulation
motif_counts_by_year_df = pd.DataFrame(list(motif_counts_by_year.items()), columns=['Year', 'Motif Count'])

# Calculate the total motifs
total_motifs = motif_counts_by_year_df['Motif Count'].sum()

# Calculate the percentage of motifs for each year
motif_counts_by_year_df['Percentage'] = (motif_counts_by_year_df['Motif Count'] / total_motifs) * 100

# Sort by year
motif_counts_by_year_df = motif_counts_by_year_df.sort_values('Year')

# Convert discord counts to a DataFrame
discord_counts_by_year_df = pd.DataFrame(list(discord_counts_by_year.items()), columns=['Year', 'Discord Count'])

# Calculate the total discords
total_discords = discord_counts_by_year_df['Discord Count'].sum()

# Calculate the percentage of discords for each year
discord_counts_by_year_df['Percentage'] = (discord_counts_by_year_df['Discord Count'] / total_discords) * 100

# Sort by year
discord_counts_by_year_df = discord_counts_by_year_df.sort_values('Year')

# Display the motif and discord results
print("Motif Counts by Year:")
print(motif_counts_by_year_df)

print("\nDiscord Counts by Year:")
print(discord_counts_by_year_df)


In [ ]:
# V1 Compute MP and return Only Motif
import pandas as pd
from collections import defaultdict
import stumpy

# Assuming your data has a 'date' column to group by year or period
# Create a dictionary to store motif occurrences by time, month, and pixel
motif_occurrences_by_month = defaultdict(list)

# Loop through the motif-neighbor pairs and count occurrences by month
for pixel_index, row in ndvi_pivot_filled.iterrows():
    ndvi_series = row.values
    dates = row.index  # Assuming the index of the row contains the dates

    window_size = 24
    mp = stumpy.stump(ndvi_series, m=window_size)

    motif_idx = mp[:, 0].argmin()
    distance = mp[motif_idx, 0]  # Matrix profile distance

    # Exclude motifs where the distance is exactly zero
    if distance > 0:
        motif_sequence = tuple(ndvi_series[motif_idx:motif_idx + window_size])
        motif_time = dates[motif_idx]  # Get the time of the motif

        # Convert motif_time to datetime object if it's a string
        if isinstance(motif_time, str):
            motif_time = pd.to_datetime(motif_time)

        motif_month = motif_time.month  # Extract the month
        motif_year = motif_time.year  # Extract the year

        # Record the motif, the pixel index where it was found, the month, and the year
        motif_occurrences_by_month[(motif_year, motif_month, motif_time)].append((pixel_index, motif_sequence))

# Filter motifs that occur in the same month and year across multiple pixels
common_motifs_by_month = {
    (year, month, time): pixels for (year, month, time), pixels in motif_occurrences_by_month.items() if len(pixels) > 1
}

# Convert the common motifs to a DataFrame for easier manipulation
common_motifs_list = []
for (year, month, time), pixels in common_motifs_by_month.items():
    for pixel_index, motif_sequence in pixels:
        common_motifs_list.append({
            'Year': year,
            'Month': month,
            'Time': time,
            'Pixel Index': pixel_index,
            'Motif Sequence': motif_sequence
        })

common_motifs_df = pd.DataFrame(common_motifs_list)

# Sort the DataFrame by year and month
common_motifs_df = common_motifs_df.sort_values(by=['Year', 'Month', 'Time'])

# Display the results
print("Common motifs observed at the same time across multiple pixels, sorted by year and month:")
print(common_motifs_df)


In [ ]:
# V2 Compute MP and return Motif & 1 Nearest Neighbours
import pandas as pd
from collections import defaultdict
import stumpy

# Dictionary to store motif occurrences with nearest neighbors by time, month, and pixel
motif_occurrences_by_month = defaultdict(list)

# Loop through the motif-neighbor pairs and count occurrences by month
for pixel_index, row in ndvi_pivot_filled.iterrows():
    ndvi_series = row.values
    dates = row.index  # Assuming the index of the row contains the dates

    window_size = 24
    mp = stumpy.stump(ndvi_series, m=window_size)

    # Identify the motif and its nearest neighbors
    motif_idx = mp[:, 0].argmin()  # Index of motif with the smallest distance
    distance = mp[motif_idx, 0]  # Matrix profile distance of the motif

    # Exclude motifs where the distance is exactly zero
    if distance > 0:
        motif_sequence = tuple(ndvi_series[motif_idx:motif_idx + window_size])
        motif_time = dates[motif_idx]  # Get the time of the motif

        # Convert motif_time to datetime if it's a string
        if isinstance(motif_time, str):
            motif_time = pd.to_datetime(motif_time)

        motif_month = motif_time.month  # Extract the month
        motif_year = motif_time.year  # Extract the year

        # Capture the indices and sequences for the nearest neighbors
        neighbors = []
        for i in range(1, 2):  # Assuming you want the two nearest neighbors
            neighbor_idx = mp[motif_idx, i]
            neighbor_sequence = tuple(ndvi_series[int(neighbor_idx):int(neighbor_idx) + window_size])
            neighbor_time = dates[int(neighbor_idx)]
            neighbors.append({
                'Neighbor Sequence': neighbor_sequence,
                'Neighbor Time': neighbor_time
            })

        # Record the motif, pixel index, month, year, and nearest neighbors
        motif_occurrences_by_month[(motif_year, motif_month, motif_time)].append({
            'Pixel Index': pixel_index,
            'Motif Sequence': motif_sequence,
            'Neighbors': neighbors
        })

# Filter motifs with nearest neighbors that occur in the same month and year across multiple pixels
common_motifs_by_month = {
    (year, month, time): pixels for (year, month, time), pixels in motif_occurrences_by_month.items() if len(pixels) > 1
}

# Convert the common motifs with neighbors to a DataFrame
common_motifs_list = []
for (year, month, time), pixels in common_motifs_by_month.items():
    for data in pixels:
        pixel_index = data['Pixel Index']
        motif_sequence = data['Motif Sequence']
        neighbors = data['Neighbors']

        # Add each neighbor to the DataFrame
        for neighbor in neighbors:
            common_motifs_list.append({
                'Year': year,
                'Month': month,
                'Time': time,
                'Pixel Index': pixel_index,
                'Motif Sequence': motif_sequence,
                'Neighbor Sequence': neighbor['Neighbor Sequence'],
                'Neighbor Time': neighbor['Neighbor Time']
            })

# Create DataFrame and sort by year and month
common_motifs_df = pd.DataFrame(common_motifs_list)
common_motifs_df = common_motifs_df.sort_values(by=['Year', 'Month', 'Time'])

# Display the results
print("Common motifs with nearest neighbors observed at the same time across multiple pixels, sorted by year and month:")
print(common_motifs_df)


In [ ]:
# V3 Compute MP and return Motif & 2 Nearest Neighbours
import pandas as pd
from collections import defaultdict
import stumpy

# Dictionary to store motif occurrences with nearest neighbors by time, month, and pixel
motif_occurrences_by_month = defaultdict(list)

# Loop through the motif-neighbor pairs and count occurrences by month
for pixel_index, row in ndvi_pivot_filled.iterrows():
    ndvi_series = row.values
    dates = row.index  # Assuming the index of the row contains the dates

    window_size = 24
    mp = stumpy.stump(ndvi_series, m=window_size)

    # Identify the motif and its nearest neighbors
    motif_idx = mp[:, 0].argmin()  # Index of motif with the smallest distance
    distance = mp[motif_idx, 0]  # Matrix profile distance of the motif

    # Exclude motifs where the distance is exactly zero
    if distance > 0:
        motif_sequence = tuple(ndvi_series[motif_idx:motif_idx + window_size])
        motif_time = dates[motif_idx]  # Get the time of the motif

        # Convert motif_time to datetime if it's a string
        if isinstance(motif_time, str):
            motif_time = pd.to_datetime(motif_time)

        motif_month = motif_time.month  # Extract the month
        motif_year = motif_time.year  # Extract the year

        # Capture the indices and sequences for the nearest neighbors
        neighbors = []
        for i in range(1, 3):  # Assuming you want the two nearest neighbors
            neighbor_idx = mp[motif_idx, i]
            neighbor_sequence = tuple(ndvi_series[int(neighbor_idx):int(neighbor_idx) + window_size])
            neighbor_time = dates[int(neighbor_idx)]
            neighbors.append({
                'Neighbor Sequence': neighbor_sequence,
                'Neighbor Time': neighbor_time
            })

        # Record the motif, pixel index, month, year, and nearest neighbors
        motif_occurrences_by_month[(motif_year, motif_month, motif_time)].append({
            'Pixel Index': pixel_index,
            'Motif Sequence': motif_sequence,
            'Neighbors': neighbors
        })

# Filter motifs with nearest neighbors that occur in the same month and year across multiple pixels
common_motifs_by_month = {
    (year, month, time): pixels for (year, month, time), pixels in motif_occurrences_by_month.items() if len(pixels) > 1
}

# Convert the common motifs with neighbors to a DataFrame
common_motifs_list = []
for (year, month, time), pixels in common_motifs_by_month.items():
    for data in pixels:
        pixel_index = data['Pixel Index']
        motif_sequence = data['Motif Sequence']
        neighbors = data['Neighbors']

        # Add each neighbor to the DataFrame
        for neighbor in neighbors:
            common_motifs_list.append({
                'Year': year,
                'Month': month,
                'Time': time,
                'Pixel Index': pixel_index,
                'Motif Sequence': motif_sequence,
                'Neighbor Sequence': neighbor['Neighbor Sequence'],
                'Neighbor Time': neighbor['Neighbor Time']
            })

# Create DataFrame and sort by year and month
common_motifs_df = pd.DataFrame(common_motifs_list)
common_motifs_df = common_motifs_df.sort_values(by=['Year', 'Month', 'Time'])

# Display the results
print("Common motifs with nearest neighbors observed at the same time across multiple pixels, sorted by year and month:")
print(common_motifs_df)


Plottings

In [ ]:
# Heatmap

import seaborn as sns
import matplotlib.pyplot as plt

# Filter for valid motifs and neighbor sequences
filtered_df = common_motifs_df[(common_motifs_df['Motif Sequence'].notna()) & (common_motifs_df['Neighbor Sequence'] != '()')]

# Count the occurrences of motifs and nearest neighbors by Year and Month
monthly_counts = filtered_df.groupby(['Year', 'Month']).size().reset_index(name='Total_Count')

# Pivot the data for heatmap compatibility
heatmap_df = monthly_counts.pivot(index='Month', columns='Year', values='Total_Count')

# Plot the heatmap
plt.figure(figsize=(8, 3))
sns.heatmap(heatmap_df, annot=True, cmap="YlGnBu", fmt=".0f", linewidths=.5)
# plt.title('Monthly Count of Motifs and Nearest Neighbors in Europe')
plt.xlabel('Year')
plt.xticks(rotation=45, fontsize=10)
plt.ylabel('Month')
plt.show()
